In [1]:
!pip install torch==2.3.0 torchvision torchaudio -f https://download.pytorch.org/whl/cu121/torch_stable.html
!pip install "unsloth[cu121-torch230] @ git+https://github.com/unslothai/unsloth.git"

Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.2 MB/s eta 0:0

In [2]:
from unsloth import FastLlamaModel
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from unsloth.chat_templates import get_chat_template

In [6]:
from datasets import load_dataset
dataset = load_dataset("DipeshChaudhary/cleanedEmpatheticDialogueInSharegptStyle", split = "train")


Generating train split:   0%|          | 0/19533 [00:00<?, ? examples/s]

NameError: name 'formatting_prompts_func' is not defined

In [58]:
unsloth_template = (
    "{{ bos_token }}"
    "You are a mental health therapist chatbot, you try to response them empathetically keeping in mind their associated emotion.\n"
    "{% for message in messages %}"
        "{% if message['role'] == 'user' %}"
            "User is feeling {{ message['emotion'] }}.\n"
            ">>> User: {{ message['content'] }}\n"
        "{% elif message['role'] == 'assistant' %}"
            "Assistant's intent in reply should be {{ message['emotion'] }}.\n"
            ">>> Assistant: {{ message['content'] }} {{ eos_token }}\n"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
        ">>> Assistant: "
    "{% endif %}"
)
unsloth_eos_token = "eos_token"

tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"emotions":"emotion","role" : "from", "content" : "value", "user" : "patient", "assistant" : "therapist"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

def formatting_prompts_func(examples):
    convos = examples["Conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


In [59]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/19533 [00:00<?, ? examples/s]

In [60]:
dataset[0]['Conversations']

[{'emotion': 'lonely',
  'from': 'patient',
  'value': 'I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.'},
 {'emotion': 'questioning',
  'from': 'therapist',
  'value': 'Was this a friend you were in love with, or just a best friend?'},
 {'emotion': 'nostalgic',
  'from': 'patient',
  'value': 'This was a best friend. I miss her.'},
 {'emotion': 'questioning',
  'from': 'therapist',
  'value': 'Where has she gone?'},
 {'emotion': 'lonely', 'from': 'patient', 'value': 'We no longer talk.'},
 {'emotion': 'questioning',
  'from': 'therapist',
  'value': 'Oh was this something that happened because of an argument?'}]

In [61]:
dataset[0]['text']

"<|begin_of_text|>You are a mental health therapist chatbot, you try to response them empathetically keeping in mind their associated emotion.\nUser is feeling lonely.>>> User: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.\n>>>Assistant's intent in reply should be questioning.>>> Assistant: Was this a friend you were in love with, or just a best friend? <|eot_id|>\nUser is feeling nostalgic.>>> User: This was a best friend. I miss her.\n>>>Assistant's intent in reply should be questioning.>>> Assistant: Where has she gone? <|eot_id|>\nUser is feeling lonely.>>> User: We no longer talk.\n>>>Assistant's intent in reply should be questioning.>>> Assistant: Oh was this something that happened because of an argument? <|eot_id|>\n"

In [62]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/19533 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [63]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 19,533 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.815700
2,2.979600
3,2.904900
4,2.946300
5,2.840300
6,2.779200
7,2.998500
8,2.916000
9,2.905100
10,2.852300


In [65]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [66]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [53]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe next number in the Fibonacci sequence would be: 13<|eot_id|>']

In [54]:
# Function to generate response
def generate_response(input_text,emotion):
    messages = [{"emotions":emotion,"from": "human", "value": input_text}]
    inputs = tokenizer.apply_chat_template(messages,
                                           tokenize = True,
                                           add_generation_prompt = True, # Must add for generation
                                           return_tensors = "pt",
                                          ).to("cuda")
    text_streamer = TextStreamer(tokenizer)
    return model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

# Example usage
conversation_history = []
while True:
    user_input = input("User: ")
    emotion=input("Emotion: ")
    if user_input.lower() == "exit":
        print("Exiting...")
        break

    # Append user message to history
    conversation_history.append({"from": "human", "value": user_input})

    # Generate response
    response = generate_response(user_input,emotion)

    # Append bot response to history
    conversation_history.append({"from": "bot", "value": response})

    # Print bot's response
#     print("Bot:", response)

User: hlo
Emotion: lonely
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hlo<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hlo! How are you today?<|eot_id|>
User: feeling low
Emotion: lonnely
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

feeling low<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm so sorry to hear that you're feeling low. Would you like to talk about what's going on? Sometimes talking about it can help.<|eot_id|>
User: due to my exam
Emotion: demotivated
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

due to my exam<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I hope you do well on your exam!<|eot_id|>
User: i am feeling like, i can't 
Emotion: demotivating
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

i am feeling like, i can't<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It sounds like you are feeling overwhelmed and unable to handle a situation.<|eot_id|>
User: yes
Emotion: a